In [1]:
# Import Libraries. 
from splinter import Browser
import pandas as pd
import html5lib
import time
import pymongo
import requests
import re

In [2]:
# Establish the Mongo Database 
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
#Use Splinter to push the view more button to extend the full page and then save the page as HTML file. Return "Done when finished"
#This is for the returns tab. 
executable_path = {'executable_path': 'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.espn.com/nfl/stats/player/_/view/special'
browser.visit(url)

flag = True
while flag:
    try:
        browser.click_link_by_text('Show More')
        time.sleep(1)
    except:
        print("Done")
        flag = False
        

with open('return.html', 'w') as f:
    f.write(browser.html)

Done


In [4]:
#Use Pandas to read the stored HTML page and convert page to dataframe. Review the data. 
page = ('return.html')
return_tables = pd.read_html(page)
return_tables

[     RK                      Name
 0     1  Cordarrelle PattersonCHI
 1     2         Brandon WilsonCIN
 2     3          Mecole HardmanKC
 3     4        Steven Sims Jr.WSH
 4     5            Jamal AgnewDET
 ..   ..                       ...
 120  89        Diontae JohnsonPIT
 121  89         Cedrick WilsonDAL
 122  89           Adam ShaheenCHI
 123  89            David MooreSEA
 124  89        Gunner OlszewskiNE
 
 [125 rows x 2 columns],
     Unnamed: 0_level_0 Unnamed: 1_level_0 Kickoffs                    Punts  \
                    POS                 GP      ATT  YDS   AVG  LNG TD   ATT   
 0                   WR                  9       19  552  29.1  102  1     0   
 1                    S                  9       12  432  36.0   92  1     0   
 2                   WR                 10       18  414  23.0   34  0     6   
 3                   WR                  9       18  399  22.2   45  0     0   
 4                   CB                  9       14  376  26.9  100  1   

In [5]:
# The HTML had 2 tables, we formed 2 data frames from the 2 tables from the HTML
returns_df = pd.DataFrame(return_tables[0]) 
returns_df1 = pd.DataFrame(return_tables[1])

In [6]:
#We combined the two data frames into one dataframe. 
combined_returns = returns_df.join(returns_df1)

C:\Users\ngjam\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [7]:
# This is cool. We see that the names and team values are combined in the table, we need to split them. 
# We first need to confirm that the Name column is a strin, so astype forces to string. 
# Next, we wanted to use regex to split the name and team. We see that the team is in all caps, so we use the camelback technique to split them)
# Since the extract only copied the team string, we needed to go back and remove the Team strings. We replaced with blanks. 

combined_returns['Name'] = combined_returns['Name'].astype(str)
combined_returns[['Team']]= combined_returns['Name'].str.extract('([A-Z][A-Z]+)',expand =True)
combined_returns['Name'] = combined_returns.apply(lambda x: x['Name'].replace(x['Team'],""), axis=1)

In [8]:
#We cleaned up the column headers by renaming them. 
combined_returns.columns = ['RK','Name','POS','GP','K-ATT','K-YDS','K-AVG','K-LNG','K-TD','P-ATT','P-YDS','P-AVG','P-LNG','P-TD','FC','Team']
combined_returns 

,RK,Name,POS,GP,K-ATT,K-YDS,K-AVG,K-LNG,K-TD,P-ATT,P-YDS,P-AVG,P-LNG,P-TD,FC,Team
0,1,Cordarrelle Patterson,WR,9,19,552,29.1,102,1,0,0,0.0,0,0,0,CHI
1,2,Brandon Wilson,S,9,12,432,36.0,92,1,0,0,0.0,0,0,0,CIN
2,3,Mecole Hardman,WR,10,18,414,23.0,34,0,6,96,16.0,36,0,4,KC
3,4,Steven Sims Jr.,WR,9,18,399,22.2,45,0,0,0,0.0,0,0,0,WSH
4,5,Jamal Agnew,CB,9,14,376,26.9,100,1,13,54,4.2,24,0,11,DET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,89,Diontae Johnson,WR,10,0,0,0.0,0,0,5,43,8.6,14,0,1,PIT
121,89,Cedrick Wilson,WR,5,0,0,0.0,0,0,2,13,6.5,13,0,2,DAL
122,89,Adam Shaheen,TE,8,1,0,0.0,0,0,0,0,0.0,0,0,0,CHI
123,89,David Moore,WR,8,0,0,0.0,0,0,1,4,4.0,4,0,0,SEA


In [9]:
# This was the work-around to split the player name and the team if we couldn't get regular expressions to work. 

# returns_teams_list=[]
# returns_names_list=[]
# for item in combined_returns:
#     if item[-3].isupper():
#         returns_teams_list.append(item[-3:])
#         returns_names_list.append(item[:-3])
#     else:
#         returns_teams_list.append(item[-2:])
#         returns_list.append(item[:-2])
# combined_returns.Name=returns_names_list
# combined_returns['Team Name']=returns_teams_list
# combined_returns

In [10]:
#Use Splinter to push the view more button to extend the full page and then save the page as HTML file. Return "Done when finished"
#This is for the kicking tab. 
executable_path = {'executable_path': 'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.espn.com/nfl/stats/player/_/view/special/stat/kicking'
browser.visit(url)
flag = True
while flag:
    try:
        browser.click_link_by_text('Show More')
        time.sleep(1)
    except:
        print("Done")
        flag = False
        

with open('kicking.html', 'w') as f:
    f.write(browser.html)

Done


In [11]:
#Use Pandas to read the stored HTML page and convert page to dataframe. Review the data. 
page = ('kicking.html')
kicking_tables = pd.read_html(page)
kicking_tables

[    RK                    Name
 0    1        Zane GonzalezARI
 1    2       Harrison ButkerKC
 2    3           Josh LamboJAX
 3    4              Wil LutzNO
 4    4              Matt GayTB
 5    6        Chris BoswellPIT
 6    7          Matt PraterDET
 7    7        Greg ZuerleinLAR
 8    7        Justin TuckerBAL
 9   10            Joey SlyeCAR
 10  10       Austin SeibertCLE
 11  12           Dan BaileyMIN
 12  12      Brandon McManusDEN
 13  14       Adam VinatieriIND
 14  14          Brett MaherDAL
 15  14          Jason MyersSEA
 16  17          Robbie GouldSF
 17  17          Mason CrosbyGB
 18  17        Randy BullockCIN
 19  17     Ka'imi FairbairnHOU
 20  21         Jake ElliottPHI
 21  21         Eddy PineiroCHI
 22  23       Dustin HopkinsWSH
 23  24          Matt BryantATL
 24  24       Daniel CarlsonOAK
 25  24        Jason SandersMIA
 26  24  Chase McLaughlinLAC/SF
 27  28     Stephen HauschkaBUF
 28  28        Aldrick RosasNYG
 29  30    Stephen GostkowskiNE
 30  30 

In [12]:
# The HTML had 2 tables, we formed 2 data frames from the 2 tables from the HTML
kicking_df = pd.DataFrame(kicking_tables[0]) 
kicking_df1 = pd.DataFrame(kicking_tables[1])

In [13]:
#We combined the two data frames into one dataframe. 
combined_kicks = kicking_df.join(kicking_df1)

In [14]:
# We first need to confirm that the Name column is a strin, so astype forces to string. 
# Next, we wanted to use regex to split the name and team. We see that the team is in all caps, so we use the camelback technique to split them)
# Since the extract only copied the team string, we needed to go back and remove the Team strings. We replaced with blanks. 
combined_kicks['Name'] = combined_kicks['Name'].astype(str)

[['Team']]= combined_kicks['Name'].str.extract('([A-Z][A-Z]+)',expand =True)
combined_kicks['Name'] = combined_kicks.apply(lambda x: x['Name'].replace(x['Team'],""), axis=1)

In [15]:
#We cleaned up the column headers by renaming them. We do not have any null data. 
combined_kicks.columns = ['RK', 'Name','POS','GP','FGM','FGA','FG%','LNG','1-19','20-29','30-39','40-49','50+','XPM','XPA','XP%','Team']
combined_kicks

,RK,Name,POS,GP,FGM,FGA,FG%,LNG,1-19,20-29,30-39,40-49,50+,XPM,XPA,XP%,Team
0,1,Zane Gonzalez,PK,10,24,27,88.9,54,0-0,9-9,10-11,3-5,2-2,18,18,100.0,ARI
1,2,Harrison Butker,PK,10,23,27,85.2,54,0-0,7-7,7-8,8-8,1-4,29,31,93.5,KC
2,3,Josh Lambo,PK,9,22,22,100.0,48,0-0,8-8,9-9,5-5,0-0,12,13,92.3,JAX
3,4,Wil Lutz,PK,9,19,23,82.6,58,1-1,6-6,4-4,6-8,2-4,19,20,95.0,NO
4,4,Matt Gay,PK,9,19,22,86.4,58,0-0,5-5,3-4,8-9,3-4,23,25,92.0,TB
5,6,Chris Boswell,PK,10,18,20,90.0,51,1-1,6-6,4-4,6-7,1-2,20,20,100.0,PIT
6,7,Matt Prater,PK,9,17,20,85.0,55,0-0,6-6,2-2,3-5,6-7,22,23,95.7,DET
7,7,Greg Zuerlein,PK,9,17,22,77.3,58,0-0,6-6,4-4,3-7,4-5,23,23,100.0,LAR
8,7,Justin Tucker,PK,9,17,17,100.0,51,0-0,7-7,4-4,5-5,1-1,31,32,96.9,BAL
9,10,Joey Slye,PK,9,16,22,72.7,55,0-0,2-2,4-4,5-8,5-8,21,23,91.3,CAR


In [16]:
# This was the work-around to split the player name and the team if we couldn't get regular expressions to work.

# kick_teams_list=[]
# kick_names_list=[]
# for item in combined_kicks.Name:
#     if item[-3].isupper():
#         kick_teams_list.append(item[-3:])
#         kick_names_list.append(item[:-3])
#     else:
#         kick_teams_list.append(item[-2:])
#         kick_names_list.append(item[:-2])
# combined_kicks.Name=punt_names_list
# combined_kicks['Team Name']=punt_teams_list
# combined_punts

In [17]:
#Use Splinter to push the view more button to extend the full page and then save the page as HTML file. Return "Done when finished"
#This is for the punting tab.

executable_path = {'executable_path': 'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.espn.com/nfl/stats/player/_/view/special/stat/punting'
browser.visit(url)

flag = True
while flag:
    try:
        browser.click_link_by_text('Show More')
        time.sleep(1)
    except:
        print("Done")
        flag = False
        

with open('punting.html', 'w') as f:
    f.write(browser.html)

Done


In [18]:
#Use Pandas to read the stored HTML page and convert page to dataframe. Review the data. 
page = ('punting.html')
punting_tables = pd.read_html(page)
punting_tables

[    RK                  Name
 0    1           Andy LeeARI
 1    2          Tress WayWSH
 2    3            Ty LongLAC
 3    4   Cameron JohnstonPHI
 4    5         Brett KernTEN
 5    6          A.J. ColeOAK
 6    7    Lachlan EdwardsNYJ
 7    8      Johnny HekkerLAR
 8    8       Jamie GillanCLE
 9   10       Jordan BerryPIT
 10  11        Riley DixonNYG
 11  12    Michael DicksonSEA
 12  13         Matt HaackMIA
 13  14            JK ScottGB
 14  15    Michael PalardyCAR
 15  16     Thomas MorsteadNO
 16  17   Britton ColquittMIN
 17  18      Pat O'DonnellCHI
 18  19        Logan CookeJAX
 19  20     Dustin ColquittKC
 20  21  Rigoberto SanchezIND
 21  22         Jake BaileyNE
 22  23    Mitch WishnowskySF
 23  24       Colby WadmanDEN
 24  25        Kevin HuberCIN
 25  26      Bradley PinionTB
 26  27         Sam MartinDET
 27  28    Corey BojorquezBUF
 28  29        Chris JonesDAL,
    POS  GP  PUNTS   YDS  LNG   AVG   NET  PBLK  IN20  TB  FC  ATT  YDS.1  \
 0    P   9     36  17

In [19]:
# The HTML had 2 tables, we formed 2 data frames from the 2 tables from the HTML
punting_df = pd.DataFrame(punting_tables[0]) 
punting_df1 = pd.DataFrame(punting_tables[1])

In [20]:
#We combined the two data frames into one dataframe. 
combined_punts = punting_df.join(punting_df1)

In [21]:
# We first need to confirm that the Name column is a strin, so astype forces to string. 
# Next, we wanted to use regex to split the name and team. We see that the team is in all caps, so we use the camelback technique to split them)
# Since the extract only copied the team string, we needed to go back and remove the Team strings. We replaced with blanks. 

combined_punts['Name'] = combined_punts['Name'].astype(str)
combined_punts[['Team']]= combined_punts['Name'].str.extract('([A-Z][A-Z]+)',expand =True)
combined_punts['Name'] = combined_punts.apply(lambda x: x['Name'].replace(x['Team'],""), axis=1)

In [22]:
#We cleaned up the column headers by renaming them. We do not have any null data. 
combined_punts.rename(columns={'YDS.1':'RTN YDS','AVG.1':'RTN AVG','ATT':'RTN ATT'}, inplace=True)
combined_punts

,RK,Name,POS,GP,PUNTS,YDS,LNG,AVG,NET,PBLK,IN20,TB,FC,RTN ATT,RTN YDS,RTN AVG,Team
0,1,Andy Lee,P,9,36,1749,64,48.6,42.2,1,11,3,5,21,128,6.1,ARI
1,2,Tress Way,P,9,44,2129,66,48.4,43.8,0,17,1,14,20,182,9.1,WSH
2,3,Ty Long,P,10,29,1383,60,47.7,41.9,0,11,0,5,22,167,7.6,LAC
3,4,Cameron Johnston,P,9,34,1620,60,47.6,43.1,0,13,2,13,15,113,7.5,PHI
4,5,Brett Kern,P,10,54,2561,70,47.4,43.3,0,28,2,13,19,184,9.7,TEN
5,6,A.J. Cole,P,9,36,1702,60,47.3,39.9,0,15,2,9,21,227,10.8,OAK
6,7,Lachlan Edwards,P,9,55,2566,63,46.7,42.6,0,19,0,8,31,221,7.1,NYJ
7,8,Johnny Hekker,P,9,34,1581,71,46.5,41.3,1,10,1,7,15,117,7.8,LAR
8,8,Jamie Gillan,P,10,46,2139,71,46.5,40.6,1,23,5,15,11,131,11.9,CLE
9,10,Jordan Berry,P,10,44,2044,62,46.5,41.5,0,15,2,14,20,179,8.9,PIT


In [23]:
# This was the work-around to split the player name and the team if we couldn't get regular expressions to work.

# punt_teams_list=[]
# punt_names_list=[]
# for item in combined_punts.Name:
#     if item[-3].isupper():
#         punt_teams_list.append(item[-3:])
#         punt_names_list.append(item[:-3])
#     else:
#         punt_teams_list.append(item[-2:])
#         punt_names_list.append(item[:-2])
# combined_punts.Name=punt_names_list
# combined_punts['Team Name']=punt_teams_list
# combined_punts

In [24]:
#Create the database. We used the same database name between all team members. 
db = client.NFL_StatsDB

In [25]:
#make a collection for the kicking dataframe. 
collections1 = db.kicks
kicks_data = combined_kicks.to_dict(orient='records')
collections1.insert_many(kicks_data)

In [26]:
#make a collections for the returns dataframe. 
collections2 = db.returns
returns_data= combined_returns.to_dict(orient='records')
collections2.insert_many(returns_data)

In [27]:
#make a collection for the punting dataframe. 
collections3 = db.punts
punts_data= combined_punts.to_dict(orient='records')
collections3.insert_many(punts_data)

In [ ]:
# URL for scraping
url = 'https://www.espn.com/nfl/stats/player/_/view/'

#setting up browser
executable_path = {'executable_path': '../chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

# while loop to move through webpage clicking show more, then grabbing the full page
flag=True
while flag:
    try:
        browser.click_link_by_text('Show More')
        time.sleep(2)
    except:
        print('cycle complete')
        flag= False
if flag==False:
    passing_tables= pd.read_html(browser.html)
    
# data cleaning: table comes as two, need to join
df=passing_tables[0]
df_2=passing_tables[1]


#join the cleaned tables
mf=df.join(df_2)
mf

#cleaning data: seperating team name from player names
#and putting them in a different column
mf['Name'] = mf['Name'].astype(str)
mf[['Team']]= mf['Name'].str.extract('([A-Z][A-Z]+)',expand =True)
mf['Name'] = mf.apply(lambda x: x['Name'].replace(x['Team'],""), axis=1)

#data cleaning: drop nulls for players with no QBR
mf['QBR']=mf['QBR'].fillna(0)

#add data into MongoDB
db=client.NFL_StatsDB
collection4=db.passing
data1 = mf.to_dict(orient='records')
collection4.insert_many(data1)

In [ ]:
#Set up browser
url = 'https://www.espn.com/nfl/stats/player/_/stat/receiving'
executable_path = {'executable_path': 'C:/Users/Daniel/Desktop/Drivers/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#loop through page to scrape data at the end
browser.visit(url)
flag=True
while flag:
    try:
        browser.click_link_by_text('Show More')
        time.sleep(1)
    except:
        print('cycle complete')
        flag= False
if flag==False:
    receiving_tables= pd.read_html(browser.html)
    
#join tables   
df=receiving_tables[0]
df_2=receiviing_tables[1] 
#join the tables
mf=df.join(df_2)

#clean the team names from the player names
mf['Name'] = mf['Name'].astype(str)
mf[['Team']]= mf['Name'].str.extract('([A-Z][A-Z]+)',expand =True)
mf['Name'] = mf.apply(lambda x: x['Name'].replace(x['Team'],""), axis=1)
#add into mongoDB
db=client.NFL_StatsDB
collection5=db.receiving
data1 = mf.to_dict(orient='records')
collection5.insert_many(data1)

In [ ]:
#setting up the browser
url = 'https://www.espn.com/nfl/stats/player/_/stat/rushing'
executable_path = {'executable_path': 'C:/Users/Daniel/Desktop/Drivers/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)
#looping throug page to collect whole table
flag=True
while flag:
    try:
        browser.click_link_by_text('Show More')
        time.sleep(1)
    except:
        print('cycle complete')
        flag= False
if flag==False:
    rushing_tables= pd.read_html(browser.html)
    
#joining table
df=rushing_tables[0]
df_2=rushing_tables[1]

#cleaning team name from player names and joining
mf=df.join(df_2)


mf['Name'] = mf['Name'].astype(str)
mf[['Team']]= mf['Name'].str.extract('([A-Z][A-Z]+)',expand =True)
mf['Name'] = mf.apply(lambda x: x['Name'].replace(x['Team'],""), axis=1)

#insert into mongoDB
db=client.NFL_StatsDB
collection6=db.rushing
data1 = mf.to_dict(orient='records')
collection6.insert_many(data1)

In [ ]:
# ESPN defense page
url = 'https://www.espn.com/nfl/stats/player/_/view/defense'

# Loads table data from ESPN website
tableDF = pd.read_html(url)
df= tableDF[0]

# The code takes in the team name as part of the player last name, this code
# extracts the team name from the last name
teams_list=[]
names_list=[]

for item in df.Name:
    if item[-3].isupper():
        teams_list.append(item[-3:])
        names_list.append(item[:-3])
    else:
        teams_list.append(item[-2:])
        names_list.append(item[:-2])
df.Name = names_list
df['Team Name']=teams_list

# Join both tables with player name data and player stat data
bothDF = tableDF[0].join(tableDF[1])

# In place rename the columns since we get multilevel columns when scraping the data
bothDF.rename(columns={bothDF.columns[2]: 'Team Name', bothDF.columns[3]: "POS", bothDF.columns[4]:"GP",
                        bothDF.columns[5]:"SOLO", bothDF.columns[6]:"AST", bothDF.columns[7]:"TOT",
                      bothDF.columns[8]:"SACK", bothDF.columns[9]:"YDS" ,
                      bothDF.columns[10]:"TFL",bothDF.columns[11]:"PD",
                      bothDF.columns[12]:"INT", bothDF.columns[13]:"YDS",
                      bothDF.columns[14]:"LNG", bothDF.columns[15]:"TD",
                      bothDF.columns[16]:"FF",bothDF.columns[17]:"FR",
                      bothDF.columns[18]:"FTD" }, inplace=True)

# Define database and collection
db = client.NFL_StatsDB
collection = db.Defense

# Converting our data to a dictionary to load it into MonboDB
data1 = bothDF.to_dict(orient='records')

# Insert our data into
collection.insert_many(data1)

In [ ]:
# ESPN scoring page
url = 'https://www.espn.com/nfl/stats/player/_/view/scoring'
# Loads table data from ESPN website
tableDF = pd.read_html(url)
df= tableDF[0]
# The code takes in the team name as part of the player last name, this code
# extracts the team name from the last name
teams_list=[]
names_list=[]

for item in df.Name:
    if item[-3].isupper():
        teams_list.append(item[-3:])
        names_list.append(item[:-3])
    else:
        teams_list.append(item[-2:])
        names_list.append(item[:-2])
df.Name = names_list
df['Team Name']=teams_list
# Join both tables with player name data and player stat data
bothDF = tableDF[0].join(tableDF[1])
# In place rename the columns since we get multilevel columns when scraping the data
bothDF.rename(columns={bothDF.columns[2]: 'Team Name', bothDF.columns[3]: "POS", bothDF.columns[4]:"GP",
                          bothDF.columns[5]:"RUSH", bothDF.columns[6]:"REC", bothDF.columns[7]:"RET",
                      bothDF.columns[8]:"TD", bothDF.columns[9]:"FG" ,
                      bothDF.columns[10]:"XP",bothDF.columns[11]:"2PT",
                      bothDF.columns[12]:"PTS", bothDF.columns[13]:"PTS/G"}, inplace=True)
# Define database and collection
db = client.NFL_StatsDB
collection = db.Scoring
# Converting our data to a dictionary to load it into MonboDB
data1 = bothDF.to_dict(orient='records')
# Insert our data into
collection.insert_many(data1)